# Goals
1. download opensource data using wget
2. move the data to where it can be read into a dataframe
3. read the data into a spark dataframe
4. clean up column names
5. have a high overview of the dataset
6. orderBy vs. sort functions
7. sort data using raw sql syntax
8. Find Maximum Row per Group in DataFrame
9.
10


In [0]:
# !pip install pyspark
# !pip install wget
# !pip install pandas

# Import libraries

In [0]:
import pandas as pd
import wget
from pyspark.sql import SparkSession

# Create a spark session

In [0]:
# instantiate spark session
spark = SparkSession.builder.appName("Kairitu_pyspark").getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

# Opensource Failed Baks Dataset

# Download the dataset

In [0]:
# Download the dataset
wget.download("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/banklist.csv")

Out[41]: 'banklist (2).csv'

# Move or copy the dataset  to /FileStore/tables/

In [0]:
# Move or copy the dataset from the /databricks/driver/ to /FileStore/tables/
# copy
# dbutils.fs.cp('file:/databricks/driver/banklist (1).csv', 'dbfs:/FileStore/tables/')
# move and rename
dbutils.fs.mv('file:/databricks/driver/banklist (2).csv', 'dbfs:/FileStore/tables/banks_failed.csv')

Out[42]: True

# Read the dataset into a dataframe

In [0]:
# Read the dataset into a dataframe
banks = spark.read.csv("/FileStore/tables/banks_failed.csv", header=True, inferSchema=True)

banks.show(n=2, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------
 Bank Name�             | First National Bank of Lindsay        
 City�                  | Lindsay                               
 State�                 | OK                                    
 Cert�                  | 4134                                  
 Acquiring Institution� | First Bank & Trust Co.                
 Closing Date�          | 18-Oct-24                             
 Fund                   | 10547                                 
-RECORD 1-------------------------------------------------------
 Bank Name�             | Republic First Bank dba Republic Bank 
 City�                  | Philadelphia                          
 State�                 | PA                                    
 Cert�                  | 27332                                 
 Acquiring Institution� | Fulton Bank, National Association     
 Closing Date�          | 26-Apr-24                             
 Fund                   |

In [0]:
banks.columns

Out[46]: ['Bank Name�',
 'City�',
 'State�',
 'Cert�',
 'Acquiring Institution�',
 'Closing Date�',
 'Fund']

In [0]:
# Fix column names by removing the "�"
for cols in banks.columns:
    banks = banks.withColumnRenamed(cols, cols.replace("�",""))

banks.columns

Out[48]: ['Bank Name',
 'City',
 'State',
 'Cert',
 'Acquiring Institution',
 'Closing Date',
 'Fund']

In [0]:
# Fix column names by replacing the spaces with "_"
for cols in banks.columns:
    banks = banks.withColumnRenamed(cols, cols.replace(" ","_"))

banks.columns

Out[49]: ['Bank_Name',
 'City',
 'State',
 'Cert',
 'Acquiring_Institution',
 'Closing_Date',
 'Fund']

In [0]:
# Fix column names by making all names lowercase
for cols in banks.columns:
    banks = banks.withColumnRenamed(cols, cols.lower())

banks.columns

Out[50]: ['bank_name',
 'city',
 'state',
 'cert',
 'acquiring_institution',
 'closing_date',
 'fund']

In [0]:
banks.show(n=2, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 bank_name             | First National Bank of Lindsay        
 city                  | Lindsay                               
 state                 | OK                                    
 cert                  | 4134                                  
 acquiring_institution | First Bank & Trust Co.                
 closing_date          | 18-Oct-24                             
 fund                  | 10547                                 
-RECORD 1------------------------------------------------------
 bank_name             | Republic First Bank dba Republic Bank 
 city                  | Philadelphia                          
 state                 | PA                                    
 cert                  | 27332                                 
 acquiring_institution | Fulton Bank, National Association     
 closing_date          | 26-Apr-24                             
 fund                  | 10546          

# dir(banks)

In [0]:
dir(banks)

Out[53]: ['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collect_as_arrow',
 '_jcols',
 '_jdf',
 '_jmap',
 '_joinAsOf',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_session',
 '_sort_cols',
 '_sql_ctx',
 '_support_repr_html',
 '_to_corrected_pandas_type',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'display',
 'distinct',
 'drop',
 'dropDuplicates',
 'drop_duplicates',
 '

In [0]:
banks.columns

Out[78]: ['bank_name',
 'city',
 'state',
 'cert',
 'acquiring_institution',
 'closing_date',
 'fund']

In [0]:
banks.summary().show()

+-------+--------------------+-------+-----+------------------+---------------------+------------+-----------------+
|summary|           bank_name|   city|state|              cert|acquiring_institution|closing_date|             fund|
+-------+--------------------+-------+-----+------------------+---------------------+------------+-----------------+
|  count|                 570|    570|  570|               570|                  570|         570|              570|
|   mean|                null|   null| null|31604.777192982456|                 null|        null|10043.09649122807|
| stddev|                null|   null| null|16490.726666011924|                 null|        null|1109.862110605227|
|    min|1st American Stat...|Acworth|   AL|                91|      1st United Bank|    1-Aug-08|             4645|
|    25%|                null|   null| null|             20078|                 null|        null|            10118|
|    50%|                null|   null| null|             32069| 

In [0]:
banks.describe().show()

+-------+--------------------+-------+-----+------------------+---------------------+------------+-----------------+
|summary|           bank_name|   city|state|              cert|acquiring_institution|closing_date|             fund|
+-------+--------------------+-------+-----+------------------+---------------------+------------+-----------------+
|  count|                 570|    570|  570|               570|                  570|         570|              570|
|   mean|                null|   null| null|31604.777192982456|                 null|        null|10043.09649122807|
| stddev|                null|   null| null|16490.726666011924|                 null|        null|1109.862110605227|
|    min|1st American Stat...|Acworth|   AL|                91|      1st United Bank|    1-Aug-08|             4645|
|    max|               ebank|Wyoming|   WY|             59017|  Your Community Bank|    9-Sep-11|            10547|
+-------+--------------------+-------+-----+------------------+-

In [0]:
banks.head(n=5)

Out[74]: [Row(bank_name='First National Bank of Lindsay', city='Lindsay', state='OK', cert=4134, acquiring_institution='First Bank & Trust Co.', closing_date='18-Oct-24', fund=10547),
 Row(bank_name='Republic First Bank dba Republic Bank', city='Philadelphia', state='PA', cert=27332, acquiring_institution='Fulton Bank, National Association', closing_date='26-Apr-24', fund=10546),
 Row(bank_name='Citizens Bank', city='Sac City', state='IA', cert=8758, acquiring_institution='Iowa Trust & Savings Bank', closing_date='3-Nov-23', fund=10545),
 Row(bank_name='Heartland Tri-State Bank', city='Elkhart', state='KS', cert=25851, acquiring_institution='Dream First Bank, N.A.', closing_date='28-Jul-23', fund=10544),
 Row(bank_name='First Republic Bank', city='San Francisco', state='CA', cert=59017, acquiring_institution='JPMorgan Chase Bank, N.A.', closing_date='1-May-23', fund=10543)]

In [0]:
banks.orderBy(['state', 'fund']).show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|   CapitalSouth Bank|  Birmingham|   AL|22130|           IBERIABANK|   21-Aug-09|10106|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|         Nexity Bank|  Birmingham|   AL|19794| AloStar Bank of C...|   15-Apr-11|10356|
|       Superior Bank|  Birmingham|   AL|17750| Superior Bank, Na...|   15-Apr-11|10358|
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
|Sinclair National...|    Gravette|   AR|34248|   Delta Trust & Bank|    7-Sep-01| 4649|
|   ANB Financial, NA

In [0]:
banks.show(n=3, truncate=False)

+-------------------------------------+------------+-----+-----+---------------------------------+------------+-----+
|bank_name                            |city        |state|cert |acquiring_institution            |closing_date|fund |
+-------------------------------------+------------+-----+-----+---------------------------------+------------+-----+
|First National Bank of Lindsay       |Lindsay     |OK   |4134 |First Bank & Trust Co.           |18-Oct-24   |10547|
|Republic First Bank dba Republic Bank|Philadelphia|PA   |27332|Fulton Bank, National Association|26-Apr-24   |10546|
|Citizens Bank                        |Sac City    |IA   |8758 |Iowa Trust & Savings Bank        |3-Nov-23    |10545|
+-------------------------------------+------------+-----+-----+---------------------------------+------------+-----+
only showing top 3 rows



In [0]:
banks.summary().show()

+-------+--------------------+-------+-----+------------------+---------------------+------------+-----------------+
|summary|           bank_name|   city|state|              cert|acquiring_institution|closing_date|             fund|
+-------+--------------------+-------+-----+------------------+---------------------+------------+-----------------+
|  count|                 570|    570|  570|               570|                  570|         570|              570|
|   mean|                null|   null| null|31604.777192982456|                 null|        null|10043.09649122807|
| stddev|                null|   null| null|16490.726666011924|                 null|        null|1109.862110605227|
|    min|1st American Stat...|Acworth|   AL|                91|      1st United Bank|    1-Aug-08|             4645|
|    25%|                null|   null| null|             20078|                 null|        null|            10118|
|    50%|                null|   null| null|             32069| 

In [0]:
banks.tail(num=5)

Out[57]: [Row(bank_name='Superior Bank, FSB', city='Hinsdale', state='IL', cert=32646, acquiring_institution='Superior Federal, FSB', closing_date='27-Jul-01', fund=6004),
 Row(bank_name='Malta National Bank', city='Malta', state='OH', cert=6629, acquiring_institution='North Valley Bank', closing_date='3-May-01', fund=4648),
 Row(bank_name='First Alliance Bank & Trust Co.', city='Manchester', state='NH', cert=34264, acquiring_institution='Southern New Hampshire Bank & Trust', closing_date='2-Feb-01', fund=4647),
 Row(bank_name='National State Bank of Metropolis', city='Metropolis', state='IL', cert=3815, acquiring_institution='Banterra Bank of Marion', closing_date='14-Dec-00', fund=4646),
 Row(bank_name='Bank of Honolulu', city='Honolulu', state='HI', cert=21029, acquiring_institution='Bank of the Orient', closing_date='13-Oct-00', fund=4645)]

# Understand the dataset

In [0]:
banks.summary().show()

+-------+--------------------+-------+-----+------------------+---------------------+------------+-----------------+
|summary|           bank_name|   city|state|              cert|acquiring_institution|closing_date|             fund|
+-------+--------------------+-------+-----+------------------+---------------------+------------+-----------------+
|  count|                 570|    570|  570|               570|                  570|         570|              570|
|   mean|                null|   null| null|31604.777192982456|                 null|        null|10043.09649122807|
| stddev|                null|   null| null|16490.726666011924|                 null|        null|1109.862110605227|
|    min|1st American Stat...|Acworth|   AL|                91|      1st United Bank|    1-Aug-08|             4645|
|    25%|                null|   null| null|             20078|                 null|        null|            10118|
|    50%|                null|   null| null|             32069| 

In [0]:
banks.show(truncate=False)

+-----------------------------------------------------+-----------------+-----+-----+-----------------------------------+------------+-----+
|bank_name                                            |city             |state|cert |acquiring_institution              |closing_date|fund |
+-----------------------------------------------------+-----------------+-----+-----+-----------------------------------+------------+-----+
|First National Bank of Lindsay                       |Lindsay          |OK   |4134 |First Bank & Trust Co.             |18-Oct-24   |10547|
|Republic First Bank dba Republic Bank                |Philadelphia     |PA   |27332|Fulton Bank, National Association  |26-Apr-24   |10546|
|Citizens Bank                                        |Sac City         |IA   |8758 |Iowa Trust & Savings Bank          |3-Nov-23    |10545|
|Heartland Tri-State Bank                             |Elkhart          |KS   |25851|Dream First Bank, N.A.             |28-Jul-23   |10544|
|First Republ

# orderBy() vs sort()

In [0]:
# OrderBy

banks.orderBy("state", ascending=True).show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
|       Superior Bank|  Birmingham|   AL|17750| Superior Bank, Na...|   15-Apr-11|10358|
|         Nexity Bank|  Birmingham|   AL|19794| AloStar Bank of C...|   15-Apr-11|10356|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|   CapitalSouth Bank|  Birmingham|   AL|22130|           IBERIABANK|   21-Aug-09|10106|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|         Allied Bank|    Mulberry|   AR|   91|         Today's Bank|   23-Sep-16|10522|
| First Southern Bank

In [0]:
banks.orderBy(['state', 'fund'], ascending=[True, False]).show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
|       Superior Bank|  Birmingham|   AL|17750| Superior Bank, Na...|   15-Apr-11|10358|
|         Nexity Bank|  Birmingham|   AL|19794| AloStar Bank of C...|   15-Apr-11|10356|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|   CapitalSouth Bank|  Birmingham|   AL|22130|           IBERIABANK|   21-Aug-09|10106|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|         Allied Bank|    Mulberry|   AR|   91|         Today's Bank|   23-Sep-16|10522|
| First Southern Bank

In [0]:
banks.orderBy(['state', 'fund'], ascending=[False, False]).show()

+--------------------+----------------+-----+-----+---------------------+------------+-----+
|           bank_name|            city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+----------------+-----+-----+---------------------+------------+-----+
|     Bank of Wyoming|     Thermopolis|   WY|22754| Central Bank & Trust|   10-Jul-09|10080|
|The First State Bank|   Barboursville|   WV|14361|       MVB Bank, Inc.|    3-Apr-20|10536|
|           Ameribank|       Northfork|   WV| 6782| The Citizens Savi...|   19-Sep-08|10014|
|Guaranty Bank, (d...|       Milwaukee|   WI|30003| First-Citizens Ba...|    5-May-17|10527|
|North Milwaukee S...|       Milwaukee|   WI|20364| First-Citizens Ba...|   11-Mar-16|10518|
|      Bank of Wausau|          Wausau|   WI|35016| Nicolet National ...|    9-Aug-13|10485|
|Banks of Wisconsi...|         Kenosha|   WI|35386| North Shore Bank,...|   31-May-13|10478|
|         Legacy Bank|       Milwaukee|   WI|34818| Seaway Bank and T.

In [0]:
banks.orderBy(['state', 'city', 'fund'], ascending=[True, True, True]).show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|   CapitalSouth Bank|  Birmingham|   AL|22130|           IBERIABANK|   21-Aug-09|10106|
|         Nexity Bank|  Birmingham|   AL|19794| AloStar Bank of C...|   15-Apr-11|10356|
|       Superior Bank|  Birmingham|   AL|17750| Superior Bank, Na...|   15-Apr-11|10358|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
| First Southern Bank|  Batesville|   AR|58052|        Southern Bank|   17-Dec-10|10322|
|   ANB Financial, NA

In [0]:
banks.sort('state', 'city', 'fund', ascending=[True, True, True]).show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|   CapitalSouth Bank|  Birmingham|   AL|22130|           IBERIABANK|   21-Aug-09|10106|
|         Nexity Bank|  Birmingham|   AL|19794| AloStar Bank of C...|   15-Apr-11|10356|
|       Superior Bank|  Birmingham|   AL|17750| Superior Bank, Na...|   15-Apr-11|10358|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
| First Southern Bank|  Batesville|   AR|58052|        Southern Bank|   17-Dec-10|10322|
|   ANB Financial, NA

In [0]:
sort_cols = ['state', 'city', 'fund']
sort_order = [True, True, True]

banks.sort(*sort_cols, ascending=[True, True, True]).show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|   CapitalSouth Bank|  Birmingham|   AL|22130|           IBERIABANK|   21-Aug-09|10106|
|         Nexity Bank|  Birmingham|   AL|19794| AloStar Bank of C...|   15-Apr-11|10356|
|       Superior Bank|  Birmingham|   AL|17750| Superior Bank, Na...|   15-Apr-11|10358|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
| First Southern Bank|  Batesville|   AR|58052|        Southern Bank|   17-Dec-10|10322|
|   ANB Financial, NA

In [0]:
from pyspark.sql.functions import col
banks.sort(col('state'), col('bank_name')).show(truncate=False)

+----------------------------------------+------------+-----+-----+-------------------------------------------+------------+-----+
|bank_name                               |city        |state|cert |acquiring_institution                      |closing_date|fund |
+----------------------------------------+------------+-----+-----+-------------------------------------------+------------+-----+
|Alabama Trust Bank, National Association|Sylacauga   |AL   |35224|Southern States Bank                       |18-May-12   |10440|
|CapitalSouth Bank                       |Birmingham  |AL   |22130|IBERIABANK                                 |21-Aug-09   |10106|
|Colonial Bank                           |Montgomery  |AL   |9609 |Branch Banking & Trust Company, (BB&T)     |14-Aug-09   |10103|
|First Lowndes Bank                      |Fort Deposit|AL   |24957|First Citizens Bank                        |19-Mar-10   |10204|
|New South Federal Savings Bank          |Irondale    |AL   |32276|Beal Bank       

In [0]:
from pyspark.sql.functions import col

banks.sort(col('state').asc(), col('bank_name').desc()).show(truncate=False)

+----------------------------------------+------------+-----+-----+-------------------------------------------+------------+-----+
|bank_name                               |city        |state|cert |acquiring_institution                      |closing_date|fund |
+----------------------------------------+------------+-----+-----+-------------------------------------------+------------+-----+
|Superior Bank                           |Birmingham  |AL   |17750|Superior Bank, National Association        |15-Apr-11   |10358|
|Nexity Bank                             |Birmingham  |AL   |19794|AloStar Bank of Commerce                   |15-Apr-11   |10356|
|New South Federal Savings Bank          |Irondale    |AL   |32276|Beal Bank                                  |18-Dec-09   |10163|
|First Lowndes Bank                      |Fort Deposit|AL   |24957|First Citizens Bank                        |19-Mar-10   |10204|
|Colonial Bank                           |Montgomery  |AL   |9609 |Branch Banking &

# sort DataFrame using raw SQL syntax.

In [0]:
banks.columns

Out[102]: ['bank_name',
 'city',
 'state',
 'cert',
 'acquiring_institution',
 'closing_date',
 'fund']

In [0]:
# Sort using spark SQL

banks.createOrReplaceTempView("failed_banks")

spark.sql("select  * from failed_banks ORDER BY state asc").show(truncate=False)

+----------------------------------------+------------+-----+-----+-------------------------------------------+------------+-----+
|bank_name                               |city        |state|cert |acquiring_institution                      |closing_date|fund |
+----------------------------------------+------------+-----+-----+-------------------------------------------+------------+-----+
|Alabama Trust Bank, National Association|Sylacauga   |AL   |35224|Southern States Bank                       |18-May-12   |10440|
|Superior Bank                           |Birmingham  |AL   |17750|Superior Bank, National Association        |15-Apr-11   |10358|
|Nexity Bank                             |Birmingham  |AL   |19794|AloStar Bank of Commerce                   |15-Apr-11   |10356|
|First Lowndes Bank                      |Fort Deposit|AL   |24957|First Citizens Bank                        |19-Mar-10   |10204|
|New South Federal Savings Bank          |Irondale    |AL   |32276|Beal Bank       

In [0]:
banks.createOrReplaceTempView("failed_banks")

spark.sql("select  * from failed_banks ORDER BY fund desc").show(truncate=False)

+-----------------------------------------------------+-----------------+-----+-----+-----------------------------------+------------+-----+
|bank_name                                            |city             |state|cert |acquiring_institution              |closing_date|fund |
+-----------------------------------------------------+-----------------+-----+-----+-----------------------------------+------------+-----+
|First National Bank of Lindsay                       |Lindsay          |OK   |4134 |First Bank & Trust Co.             |18-Oct-24   |10547|
|Republic First Bank dba Republic Bank                |Philadelphia     |PA   |27332|Fulton Bank, National Association  |26-Apr-24   |10546|
|Citizens Bank                                        |Sac City         |IA   |8758 |Iowa Trust & Savings Bank          |3-Nov-23    |10545|
|Heartland Tri-State Bank                             |Elkhart          |KS   |25851|Dream First Bank, N.A.             |28-Jul-23   |10544|
|First Republ

# Find Maximum Row per Group in DataFrame

In [0]:
banks.columns

Out[105]: ['bank_name',
 'city',
 'state',
 'cert',
 'acquiring_institution',
 'closing_date',
 'fund']

In [0]:
# Imports
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

# Define a window specification to partition by the state column 
# and order by the fund column
windowDept = Window.partitionBy("state").orderBy(col("fund").desc())
# Use the window function to rank rows within each partition
rank = banks.withColumn("row",row_number().over(windowDept))
rank.show()

+--------------------+------------+-----+-----+---------------------+------------+-----+---+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|row|
+--------------------+------------+-----+-----+---------------------+------------+-----+---+
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|  1|
|       Superior Bank|  Birmingham|   AL|17750| Superior Bank, Na...|   15-Apr-11|10358|  2|
|         Nexity Bank|  Birmingham|   AL|19794| AloStar Bank of C...|   15-Apr-11|10356|  3|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|  4|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|  5|
|   CapitalSouth Bank|  Birmingham|   AL|22130|           IBERIABANK|   21-Aug-09|10106|  6|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|  7|
|         Allied Bank|    Mulberry|   AR|   91|         Today's Bank| 

In [0]:
# Filter the DataFrame to keep only the rows with rank 1 
# (i.e., the maximum fund within each state)
rank.filter(col("row") == 1).show()

+--------------------+-----------------+-----+-----+---------------------+------------+-----+---+
|           bank_name|             city|state| cert|acquiring_institution|closing_date| fund|row|
+--------------------+-----------------+-----+-----+---------------------+------------+-----+---+
|Alabama Trust Ban...|        Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|  1|
|         Allied Bank|         Mulberry|   AR|   91|         Today's Bank|   23-Sep-16|10522|  1|
|Sunrise Bank of A...|          Phoenix|   AZ|34707| First Fidelity Ba...|   23-Aug-13|10487|  1|
| First Republic Bank|    San Francisco|   CA|59017| JPMorgan Chase Ba...|    1-May-23|10543|  1|
|        Premier Bank|           Denver|   CO|34112| United Fidelity B...|   10-Jul-15|10515|  1|
|The Community's Bank|       Bridgeport|   CT|57041|          No Acquirer|   13-Sep-13|10489|  1|
|First City Bank o...|Fort Walton Beach|   FL|16748| United Fidelity B...|   16-Oct-20|10537|  1|
|The Woodbury Bank..

In [0]:
# Putting it all together -- I need the highest fund per state

# Imports
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

# Define a window specification to partition by the state column 
# and order by the fund column
windowDept = Window.partitionBy("state").orderBy(col("fund").desc())
# Use the window function to rank rows within each partition
rank = banks.withColumn("row",row_number().over(windowDept))
# Filter the DataFrame to keep only the rows with rank 1 
# (i.e., the maximum fund within each state)
rank.filter(col("row") == 1).drop("row").show()

+--------------------+-----------------+-----+-----+---------------------+------------+-----+
|           bank_name|             city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+-----------------+-----+-----+---------------------+------------+-----+
|Alabama Trust Ban...|        Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
|         Allied Bank|         Mulberry|   AR|   91|         Today's Bank|   23-Sep-16|10522|
|Sunrise Bank of A...|          Phoenix|   AZ|34707| First Fidelity Ba...|   23-Aug-13|10487|
| First Republic Bank|    San Francisco|   CA|59017| JPMorgan Chase Ba...|    1-May-23|10543|
|        Premier Bank|           Denver|   CO|34112| United Fidelity B...|   10-Jul-15|10515|
|The Community's Bank|       Bridgeport|   CT|57041|          No Acquirer|   13-Sep-13|10489|
|First City Bank o...|Fort Walton Beach|   FL|16748| United Fidelity B...|   16-Oct-20|10537|
|The Woodbury Bank...|         Woodbury|   GA|11297|        

In [0]:
# Putting it all together -- I need the Lowest fund per state

# Imports
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

# Define a window specification to partition by the state column 
# and order by the fund column
windowDept = Window.partitionBy("state").orderBy(col("fund").asc())
# Use the window function to rank rows within each partition
rank = banks.withColumn("row",row_number().over(windowDept))
# Filter the DataFrame to keep only the rows with rank 1 
# (i.e., the maximum fund within each state)
rank.filter(col("row") == 1).drop("row").show()

+--------------------+----------------+-----+-----+---------------------+------------+-----+
|           bank_name|            city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+----------------+-----+-----+---------------------+------------+-----+
|       Colonial Bank|      Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|Sinclair National...|        Gravette|   AR|34248|   Delta Trust & Bank|    7-Sep-01| 4649|
|        NextBank, NA|         Phoenix|   AZ|22314|          No Acquirer|    7-Feb-02| 4653|
|Southern Pacific ...|        Torrance|   CA|27094|            Beal Bank|    7-Feb-03| 4660|
|Colorado National...|Colorado Springs|   CO|18896|         Herring Bank|   20-Mar-09|10045|
|Connecticut Bank ...|        Stamford|   CT|19183|   Hudson United Bank|   26-Jun-02| 4656|
|   Hamilton Bank, NA|           Miami|   FL|24382| Israel Discount B...|   11-Jan-02| 4650|
|AmTrade Internati...|         Atlanta|   GA|33784|          No Acquir

In [0]:
# Putting it all together -- I need the Lowest fund per city in every state

# Imports
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

# Define a window specification to partition by the state column 
# and order by the fund column
windowDept = Window.partitionBy("state", "city").orderBy(col("fund").asc())
# Use the window function to rank rows within each partition
rank = banks.withColumn("row",row_number().over(windowDept))

# Filter the DataFrame to keep only the rows with rank 1 
# (i.e., the maximum fund within each state)
rank.filter(col("row") == 1).drop("row").show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|   CapitalSouth Bank|  Birmingham|   AL|22130|           IBERIABANK|   21-Aug-09|10106|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
| First Southern Bank|  Batesville|   AR|58052|        Southern Bank|   17-Dec-10|10322|
|   ANB Financial, NA| Bentonville|   AR|33901| Pulaski Bank and ...|    9-May-08|10005|
|Sinclair National...|    Gravette|   AR|34248|   Delta Trust & Bank|    7-Sep-01| 4649|
|         Allied Bank

In [0]:
# Putting it all together -- I need the highest fund per city in every state

# Imports
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

# Define a window specification to partition by the state column 
# and order by the fund column
windowDept = Window.partitionBy("state", "city").orderBy(col("fund").desc())
# Use the window function to rank rows within each partition
rank = banks.withColumn("row",row_number().over(windowDept))

# Filter the DataFrame to keep only the rows with rank 1 
# (i.e., the maximum fund within each state)
rank.filter(col("row") == 1).drop("row").show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|       Superior Bank|  Birmingham|   AL|17750| Superior Bank, Na...|   15-Apr-11|10358|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
| First Southern Bank|  Batesville|   AR|58052|        Southern Bank|   17-Dec-10|10322|
|   ANB Financial, NA| Bentonville|   AR|33901| Pulaski Bank and ...|    9-May-08|10005|
|Sinclair National...|    Gravette|   AR|34248|   Delta Trust & Bank|    7-Sep-01| 4649|
|         Allied Bank

# Aggregation Functions
1. Using max() Aggregate Function
2. Using min() Aggregate Function
3. Using avg() Aggregate Function

In [0]:
banks.columns

Out[127]: ['bank_name',
 'city',
 'state',
 'cert',
 'acquiring_institution',
 'closing_date',
 'fund']

# approx_count_distinct

In [0]:
# count the number of distinct bank names from the dataset
from pyspark.sql.functions import approx_count_distinct

banks.select(approx_count_distinct("bank_name")).collect()[0][0]

Out[137]: 541

In [0]:
# count the number of distinct bank names from the dataset
from pyspark.sql.functions import approx_count_distinct

banks.select(approx_count_distinct("state")).collect()[0][0]

Out[139]: 46

# avg() | mean()

In [0]:
# Using avg() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import col, max, mean, avg

# Define a window specification to partition by the state column 
# and order by the fund column
windowSpec = Window.partitionBy("state")

# Add a column for the avg fund within each state
banks_avg_fund = banks.withColumn("avg_fund", mean(col("fund")).over(windowSpec))

# Filter the DataFrame to retain only rows where the fund matches the avg fund within each state
result = banks_avg_fund.orderBy('state').filter(col("fund") == col("avg_fund"))#.drop("avg_fund")

# Show the result
result.show(n=50, truncate=False)

+-------------------------------+-----------+-----+-----+-----------------------------------+------------+-----+--------+
|bank_name                      |city       |state|cert |acquiring_institution              |closing_date|fund |avg_fund|
+-------------------------------+-----------+-----+-----+-----------------------------------+------------+-----+--------+
|Bank of Honolulu               |Honolulu   |HI   |21029|Bank of the Orient                 |13-Oct-00   |4645 |4645.0  |
|Butler Bank                    |Lowell     |MA   |26619|People's United Bank               |16-Apr-10   |10211|10211.0 |
|First Alliance Bank & Trust Co.|Manchester |NH   |34264|Southern New Hampshire Bank & Trust|2-Feb-01    |4647 |4647.0  |
|BankFirst                      |Sioux Falls|SD   |34103|Alerus Financial, N.A.             |17-Jul-09   |10081|10081.0 |
|Bank of Wyoming                |Thermopolis|WY   |22754|Central Bank & Trust               |10-Jul-09   |10080|10080.0 |
+-----------------------

# Using max() Aggregate Function

In [0]:
# Using max() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import col, max

# Define a window specification to partition by the state column 
# and order by the fund column
windowSpec = Window.partitionBy("state")

# Add a column for the maximum fund within each state
banks_max_fund = banks.withColumn("max_fund", max(col("fund")).over(windowSpec))

# Filter the DataFrame to retain only rows where the fund matches the maximum fund within each state
result = banks_max_fund.filter(col("fund") == col("max_fund")).drop("max_fund")

# Show the result
result.show(truncate=False)

+----------------------------------------+-----------------+-----+-----+------------------------------------------+------------+-----+
|bank_name                               |city             |state|cert |acquiring_institution                     |closing_date|fund |
+----------------------------------------+-----------------+-----+-----+------------------------------------------+------------+-----+
|Alabama Trust Bank, National Association|Sylacauga        |AL   |35224|Southern States Bank                      |18-May-12   |10440|
|Allied Bank                             |Mulberry         |AR   |91   |Today's Bank                              |23-Sep-16   |10522|
|Sunrise Bank of Arizona                 |Phoenix          |AZ   |34707|First Fidelity Bank, National Association |23-Aug-13   |10487|
|First Republic Bank                     |San Francisco    |CA   |59017|JPMorgan Chase Bank, N.A.                 |1-May-23    |10543|
|Premier Bank                            |Denver       

# min()

In [0]:
# Using min() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import col, min

# Define a window specification to partition by the state column 
# and order by the fund column
windowSpec = Window.partitionBy("state")

# Add a column for the minimum fund within each state
banks_min_fund = banks.withColumn("lowest_fund", min(col("fund")).over(windowSpec))

# Filter the DataFrame to retain only rows where the fund matches the maximum fund within each state
result = banks_min_fund.filter(col("fund") == col("lowest_fund")).drop("lowest_fund")

# Show the result
result.show()

+--------------------+----------------+-----+-----+---------------------+------------+-----+
|           bank_name|            city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+----------------+-----+-----+---------------------+------------+-----+
|       Colonial Bank|      Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|Sinclair National...|        Gravette|   AR|34248|   Delta Trust & Bank|    7-Sep-01| 4649|
|        NextBank, NA|         Phoenix|   AZ|22314|          No Acquirer|    7-Feb-02| 4653|
|Southern Pacific ...|        Torrance|   CA|27094|            Beal Bank|    7-Feb-03| 4660|
|Colorado National...|Colorado Springs|   CO|18896|         Herring Bank|   20-Mar-09|10045|
|Connecticut Bank ...|        Stamford|   CT|19183|   Hudson United Bank|   26-Jun-02| 4656|
|   Hamilton Bank, NA|           Miami|   FL|24382| Israel Discount B...|   11-Jan-02| 4650|
|AmTrade Internati...|         Atlanta|   GA|33784|          No Acquir

In [0]:
# Using max() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import col, max

# Define a window specification to partition by the state column 
# and order by the fund column
windowSpec = Window.partitionBy("state", "city")

# Add a column for the maximum fund within each state
banks_max_fund = banks.withColumn("max_fund", max(col("fund")).over(windowSpec))

# Filter the DataFrame to retain only rows where the fund matches the maximum fund within each state
result = banks_max_fund.filter(col("fund") == col("max_fund")).drop("max_fund")

# Show the result
result.show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|       Superior Bank|  Birmingham|   AL|17750| Superior Bank, Na...|   15-Apr-11|10358|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
| First Southern Bank|  Batesville|   AR|58052|        Southern Bank|   17-Dec-10|10322|
|   ANB Financial, NA| Bentonville|   AR|33901| Pulaski Bank and ...|    9-May-08|10005|
|Sinclair National...|    Gravette|   AR|34248|   Delta Trust & Bank|    7-Sep-01| 4649|
|         Allied Bank

In [0]:
# Using min() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import col, min

# Define a window specification to partition by the state and city column 
# and order by the fund column
windowSpec = Window.partitionBy("state", "city")

# Add a column for the minimum fund within each state
banks_min_fund = banks.withColumn("lowest_fund", min(col("fund")).over(windowSpec))

# Filter the DataFrame to retain only rows where the fund matches the maximum fund within each state
result = banks_min_fund.filter(col("fund") == col("lowest_fund")).drop("lowest_fund")

# Show the result
result.show()

+--------------------+------------+-----+-----+---------------------+------------+-----+
|           bank_name|        city|state| cert|acquiring_institution|closing_date| fund|
+--------------------+------------+-----+-----+---------------------+------------+-----+
|   CapitalSouth Bank|  Birmingham|   AL|22130|           IBERIABANK|   21-Aug-09|10106|
|  First Lowndes Bank|Fort Deposit|   AL|24957|  First Citizens Bank|   19-Mar-10|10204|
|New South Federal...|    Irondale|   AL|32276|            Beal Bank|   18-Dec-09|10163|
|       Colonial Bank|  Montgomery|   AL| 9609| Branch Banking & ...|   14-Aug-09|10103|
|Alabama Trust Ban...|   Sylacauga|   AL|35224| Southern States Bank|   18-May-12|10440|
| First Southern Bank|  Batesville|   AR|58052|        Southern Bank|   17-Dec-10|10322|
|   ANB Financial, NA| Bentonville|   AR|33901| Pulaski Bank and ...|    9-May-08|10005|
|Sinclair National...|    Gravette|   AR|34248|   Delta Trust & Bank|    7-Sep-01| 4649|
|         Allied Bank

In [0]:
# Using sum() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import col, min, sum

# Define a window specification to partition by the state and city column 
# and order by the fund column
windowSpec = Window.partitionBy("state", "city")

# Add a column for the minimum fund within each state
total_funds_state_city = banks.withColumn("total_funds", sum(col("fund")).over(windowSpec))

# Filter the DataFrame to retain only rows where the fund matches the maximum fund within each state
result = total_funds_state_city.sort(col('state').asc(), col('city').asc()).filter(col("fund") == col("total_funds")).drop("total_funds")

# Show the result
result.show(truncate = False)

+----------------------------------------+------------+-----+-----+-------------------------------------------+------------+-----+
|bank_name                               |city        |state|cert |acquiring_institution                      |closing_date|fund |
+----------------------------------------+------------+-----+-----+-------------------------------------------+------------+-----+
|First Lowndes Bank                      |Fort Deposit|AL   |24957|First Citizens Bank                        |19-Mar-10   |10204|
|New South Federal Savings Bank          |Irondale    |AL   |32276|Beal Bank                                  |18-Dec-09   |10163|
|Colonial Bank                           |Montgomery  |AL   |9609 |Branch Banking & Trust Company, (BB&T)     |14-Aug-09   |10103|
|Alabama Trust Bank, National Association|Sylacauga   |AL   |35224|Southern States Bank                       |18-May-12   |10440|
|First Southern Bank                     |Batesville  |AR   |58052|Southern Bank   

# first() and last()

In [0]:
banks.show(n=1)

+--------------------+-------+-----+----+---------------------+------------+-----+
|           bank_name|   city|state|cert|acquiring_institution|closing_date| fund|
+--------------------+-------+-----+----+---------------------+------------+-----+
|First National Ba...|Lindsay|   OK|4134| First Bank & Trus...|   18-Oct-24|10547|
+--------------------+-------+-----+----+---------------------+------------+-----+
only showing top 1 row



In [0]:
# Using sum() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import first, last

banks.select(first("bank_name")).show(truncate=False)

+------------------------------+
|first(bank_name)              |
+------------------------------+
|First National Bank of Lindsay|
+------------------------------+



In [0]:
banks.tail(num=1)

Out[168]: [Row(bank_name='Bank of Honolulu', city='Honolulu', state='HI', cert=21029, acquiring_institution='Bank of the Orient', closing_date='13-Oct-00', fund=4645)]

In [0]:
# Using sum() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import first, last

banks.select(last("bank_name")).show(truncate=False)

+----------------+
|last(bank_name) |
+----------------+
|Bank of Honolulu|
+----------------+



In [0]:
# Using sum() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import first, last, collect_list

listed_funds = banks.select(collect_list("fund"))
listed_banks= banks.select(collect_list("bank_name"))


In [0]:
listed_banks.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# pyspark window aggregate functions

In [0]:
# Using sum() Aggregate Function
from pyspark.sql import Window
from pyspark.sql.functions import col, avg, sum, min, max, row_number

# Define a window specification to partition by the state and city column 
# and order by the fund column
windowSpecAgg = Window.partitionBy("state").orderBy('state')

columns_to_select = ['state', 'funds_average', 'funds_sum',"funds_min", 'funds_max'  ]

summary = banks.withColumn("row", row_number().over(windowSpecAgg))\
    .withColumn("avg", avg(col('fund')).over(windowSpecAgg))\
        .withColumn("sum", sum(col('fund')).over(windowSpecAgg))\
            .withColumn("min", min(col('fund')).over(windowSpecAgg))\
                .withColumn("max", max(col('fund')).over(windowSpecAgg))\
                    .filter(col("row") == 1)\
                        .drop("row")
summary.show(truncate = False)

+----------------------------------------+-----------------+-----+-----+------------------------------------------+------------+-----+------------------+------+-----+-----+
|bank_name                               |city             |state|cert |acquiring_institution                     |closing_date|fund |avg               |sum   |min  |max  |
+----------------------------------------+-----------------+-----+-----+------------------------------------------+------------+-----+------------------+------+-----+-----+
|Alabama Trust Bank, National Association|Sylacauga        |AL   |35224|Southern States Bank                      |18-May-12   |10440|10247.142857142857|71730 |10103|10440|
|Allied Bank                             |Mulberry         |AR   |91   |Today's Bank                              |23-Sep-16   |10522|8874.5            |35498 |4649 |10522|
|Sunrise Bank of Arizona                 |Phoenix          |AZ   |34707|First Fidelity Bank, National Association |23-Aug-13   |10487|9

# round average column to 2 dec places


In [0]:
from pyspark.sql import functions as f

rounded_avg = summary.withColumn("avg", f.round("avg", 2))
rounded_avg.show(truncate=False)

+----------------------------------------+-----------------+-----+-----+------------------------------------------+------------+-----+--------+------+-----+-----+
|bank_name                               |city             |state|cert |acquiring_institution                     |closing_date|fund |avg     |sum   |min  |max  |
+----------------------------------------+-----------------+-----+-----+------------------------------------------+------------+-----+--------+------+-----+-----+
|Alabama Trust Bank, National Association|Sylacauga        |AL   |35224|Southern States Bank                      |18-May-12   |10440|10247.14|71730 |10103|10440|
|Allied Bank                             |Mulberry         |AR   |91   |Today's Bank                              |23-Sep-16   |10522|8874.5  |35498 |4649 |10522|
|Sunrise Bank of Arizona                 |Phoenix          |AZ   |34707|First Fidelity Bank, National Association |23-Aug-13   |10487|9930.06 |158881|4653 |10487|
|First Republic Bank  